In [101]:
import networkx as nx
import gephistreamer as gs
from gephistreamer import graph, streamer

In [5]:
from officelib.xllib import *
xl = Excel()
wb = xl.Workbooks("Helix Config.xlsx")
ws = wb.Worksheets("Schema2")
cells = ws.Cells
cr = cells.Range

In [6]:
def extract_links(ws):
    cells = ws.Cells
    cr = cells.Range
    c1 = cr("B2")
    c2 = c1.End(xlc.xlDown).End(xlc.xlToRight)
    return cr(c1, c2).Value2

In [77]:
class Item:
    def __init__(self, name):
        self.name = name
        
    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        if not isinstance(other, Item):
            return NotImplemented
        return self.name == other.name
    
    def __repr__(self):
        return "Item(%r)"%self.name
    
    def __str__(self):
        return self.name
    
def CreateItem(it):
    item = cache.get(it, None)
    if item is None:
        item = Item(it)
        cache[it] = item
    return item
        
class ItemSet:
    def __init__(self, *items):
        self.items = set()
        for it in items:
            self.add_item(it)
            
    def add_item(self, it):
        CreateItem(it)
        self.items.add(it)
        
    def recursive_yield(self):
        for it in self.items:
            v = cache[it]
            if isinstance(v, Item):
                yield v
            elif isinstance(v, ItemSet):
                yield from v.recursive_yield()
                
    def __repr__(self):
        return "ItemSet(%s)" % repr(self.items).strip("{}")
        
        
class ItemCache():
    def __init__(self):
        self._cache = {}
        
    def recursive_lookup(self, item):
        it = self._cache[item]
        if isinstance(it, ItemSet):
            yield from it.recursive_yield()
        elif isinstance(it, Item):
            yield it
            
    def __getitem__(self, key):
        return self._cache[key]
    
    def __setitem__(self, key, value):
        self._cache[key] = value
        
    def get(self, key, default=None):
        return self._cache.get(key, default)
        
    def add_item(self, it):
        item = CreateItem(it)
        self[it] = item
        

        
        
cache = ItemCache()
        
cache["Level 1"] = ItemSet("User Story", "Market Requirement", "User Requirement", "Compliance Requirement",
                      "Business Requirement", "Product Requirement")
cache["Level 2"] = ItemSet("System Requirement", "Functional Requirement", "Performance Requirement")
cache["Level 3"] = ItemSet("Design Specification", "Hardware Requirement", "Software Requirement", "Design Metric",
                           "Manufacturing Requirement", "Configuration Requirement")
cache["Level 4"] = ItemSet("Implementation Detail")
cache.add_item("User Info")
cache.add_item("Use Case")
cache.add_item("Planning")
cache.add_item("Procedure")
cache.add_item("Risk")
cache.add_item("Prototype")
cache.add_item("FMEA")

cache["Artifact"] = ItemSet("Level 1", "Level 2", "Level 3", "Level 4", "User Info", "Use Case", "Planning", "Procedure", "Risk",
                           "Prototype", "FMEA")

cache.add_item("Design Defect")
cache.add_item("Change Request")
cache.add_item("Feature Request")
cache.add_item("Work Item")
cache.add_item("Document")
cache.add_item("Test Run")
cache.add_item("Test Case")

cache["Any Info"] = ItemSet("Procedure", "General Info", "Appendices", "Definitions")

In [116]:
import itertools


def _parse_links(links):
    for src, _, tgt, link, cond in links:
        src = parse_string(src)
        tgt = parse_string(tgt)
        link = parse_string(link)
        
        src_list = list(itertools.chain.from_iterable(cache.recursive_lookup(s) for s in src))
        tgt_list = list(itertools.chain.from_iterable(cache.recursive_lookup(s) for s in tgt))
        for s in src_list:
            for t in tgt_list:
                for l in link:
                    if l == "Master Artifact Copy" and s != t:
                        continue
                    yield s, t, l, cond
    
def parse_links(links):
    return list(_parse_links(links))
    
def parse_string(string):
    return [s.strip() for s in string.strip("{}").split(",")]

In [117]:
def make_schema3(links):
    ws = wb.Worksheets("Schema3(autogen)")
    cells = ws.Cells
    cr = cells.Range
    ws.UsedRange.Clear()
    data = [("Source", "", "Target", "Link", "If")]
    data.extend((s.name, "'-->", t.name, l, c) for s, t, l, c in links)
    c1 = cr("B1")
    c2 = c1.GetOffset(len(data) - 1, len(data[0]) - 1)
    cr(c1, c2).Value2 = data
    return data

In [118]:
links = extract_links(wb.Worksheets("Schema4"))
links2 = parse_links(links)
make_schema3(links2)

[('Source', '', 'Target', 'Link', 'If'),
 ('User Story',
  "'-->",
  'System Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('User Story',
  "'-->",
  'Functional Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('User Story',
  "'-->",
  'Performance Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('Compliance Requirement',
  "'-->",
  'System Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('Compliance Requirement',
  "'-->",
  'Functional Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('Compliance Requirement',
  "'-->",
  'Performance Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('Business Requirement',
  "'-->",
  'System Requirement',
  'SME Translation',
  'Source directly translated to Target'),
 ('Business Requirement',
  "'-->",
  'Functional Requirement',
  'SME Translation',
  'Source di

In [73]:
l1 = list(cache.recursive_lookup("Level 1"))
l2 = list(cache.recursive_lookup("Level 2"))
l3 = list(cache.recursive_lookup("Level 3"))

data = []

for l in (l1, l2, l3):
    for a in l:
        for b in l:
            if a != b:
                data.append(("{%s}"%a.name, "'-->", "{%s}"%b.name, "{Related Requirements}"))
            
cells = wb.Worksheets("Schema5")
cr = cells.Range
c1 = cr("B3")
c2 = c1.GetOffset(len(data)-1, len(data[0])-1)
cr(c1, c2).Value2 = data

In [114]:
links = extract_links(wb.Worksheets("Schema3(autogen)"))
links2 = parse_links(links)
G = nx.DiGraph()

for src, tgt, link, condition in links2:
    if link == "Master Artifact Copy":
        continue
    G.add_edge(src.name, tgt.name, name=link, description=condition)

In [109]:
# Gephi Stuff
webs = gs.streamer.GephiWS(hostname="localhost", port=12346, workspace="workspace1")
stream = gs.streamer.Streamer(webs)

In [115]:
for n in G.nodes():
    node = graph.Node(n)
    stream.add_node(node)
    
for a,b in G.edges():
    d = G.get_edge_data(a,b)
    edge = graph.Edge(a,b, **d)
    stream.add_edge(edge)